In [2]:
from mongoengine import *
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime

使用** mongoengine **  
[connect](http://docs.mongoengine.org/guide/connecting.html#guide-connecting)进行数据库连接

In [3]:
connect('typhoon')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

## 1 读取文件

### 读取方式1：使用pandas

In [4]:
targetfilename=r"CH1956BST.txt"
# mac 16
# targetpath=r"/Users/casablanca/03project/typhoonSearchSys/demo_data"
# mac 15
targetpath=r"/Users/liusihan/Documents/01project/TyphoonSearchSys/demo_data"
fullname=os.path.join(targetpath,targetfilename)

In [5]:
fullname

'/Users/liusihan/Documents/01project/TyphoonSearchSys/demo_data/CH1956BST.txt'

不使用pandas的read_table

In [6]:
with open(fullname,'rb') as f:
    data=pd.read_table(f,sep='\s+',encoding='utf-8',header=None,infer_datetime_format=False)
    print('读取成功')

读取成功


--------

## 2 判断标志位的方式（必须同时满足），并获取标志位所在位置的数组（mark_indexs）

1-第一位是否为66666

2- 第7位是否不为Nan

In [7]:
data.iloc[0][0]==66666 and data.iloc[0][7]!=np.nan

True

### 写成方法：

In [8]:
def checkMark(index):
    '''
        判断是否为标志位
    '''
    return data.iloc[index][0]==66666 and data.iloc[index][7]!=np.nan

找到标志位所在的位置

In [9]:
mark_indexs=[]
for i in range(len(data)):
#     print(i)
    if checkMark(i):
        mark_indexs.append(i)

--------

## 3 获取观测值并写入mongoDB（暂时先写在list中）

In [10]:
# 保存起止数组的数组
list_startend=[]
index=0
for val in mark_indexs:
#     list_startend.append(mark_indexs[index:2])
#     print(mark_indexs[index:index+2])
    list_startend.append(mark_indexs[index:index+2])
    index=index+1
#     print(index)

[[789, 830], [830]]  
注意最后的数组是长度为1的数组，因为最后一次出现标志位后出现的都是观测数据，便结束了

----

## 4 使用mongoEngine

In [11]:
from mongoengine import *

In [12]:
connect('typhoon')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

定义 mongodb中的存储对象（ORM）

In [13]:
class GeoTyphoonRealData(Document):
    '''
        支持geojson的存储至mongodb的model
    '''
    code=StringField(max_length=10)
    num=IntField()
    date=DateTimeField()
    bp=FloatField()
    wsm=FloatField()
    # 注意此处与django中的类型不同，django的类型为IntegerField，mongoengine为IntField！
    level=IntField()
    # latlon=models.ForeignKey(Point,on_delete=models.CASCADE)
    latlon=PointField()
    meta = {'collection': 'geotyphoonrealdata'}

In [14]:
import datetime

** 提示错误如下 **:  
NotUniqueError: Tried to save duplicate unique keys (E11000 duplicate key error collection: typhoon.Typhoon_geotyphoonrealdata index: __primary_key__ dup key: { : null })  
是mongodb中的对应document中可能设置了__primary_key__，我的解决办法是 ** 删除掉 ** 

### S1 根据标志位遍历df

循环遍历list_startend

In [14]:
for val in list_startend:
    if len(val)>1:        
        print(f"起始{val[0]}——终止{val[1]-1}")
    elif len(val)==1:
        print(f"起始{val[0]}")

起始0——终止10
起始11——终止58
起始59——终止80
起始81——终止117
起始118——终止126
起始127——终止148
起始149——终止158
起始159——终止168
起始169——终止186
起始187——终止196
起始197——终止218
起始219——终止223
起始224——终止270
起始271——终止286
起始287——终止298
起始299——终止309
起始310——终止323
起始324——终止335
起始336——终止371
起始372——终止384
起始385——终止396
起始397——终止428
起始429——终止463
起始464——终止504
起始505——终止520
起始521——终止535
起始536——终止586
起始587——终止622
起始623——终止659
起始660——终止680
起始681——终止697
起始698——终止744
起始745——终止803
起始804——终止817
起始818——终止831
起始832——终止850
起始851——终止865
起始866


In [15]:
def getHeaderBody(start,end=None):
    '''
        根据df的起止位置截取其中的头部（66666	0	25	0	1	0	6.0	(nameless)	20180501.0），
        以及数据内容body（2017041406	0	109	1302	1010	10）
    '''
    header=data.iloc[start-1]
    body=data.iloc[start:end,0:6]
    return header,body

In [16]:
from datetime import datetime

- 19-04-03 此处加入一个新的需求，加入对于 **台风 数字编号 **的获取

In [19]:
data.iloc[0]

0          66666
1              0
2             10
3              1
4              0
5              0
6              6
7     (nameless)
8    2.01107e+07
Name: 0, dtype: object

In [17]:
def convert2Typhoon(obj,code):
    '''
        根据传入的series将其转为typhoob Model
    '''
    lat=float(f"{str(obj[2])[:-1]}.{str(obj[2])[-1:]}")
    lon=float(f"{str(obj[3])[:-1]}.{str(obj[3])[-1:]}")
    stamp_str=obj[0]
    stamp=datetime.strptime(str(stamp_str),'%Y%m%d%H%M')
    typhoon_temp=GeoTyphoonRealData(code=code,
                                date=stamp,
                                bp=obj[4],
                                wsm=obj[5],
                                level=obj[1],
                                latlon=[lon,lat])
    return typhoon_temp

### S2 拼接经纬度
获取经纬度：  
    - 将字符串转成float，并且最后一位为小数点后一位

### S3 获取当前时间

### S4 写入mongodb

### S5 遍历起始list，自动获取header以及body

In [18]:
typhoon_list=[]
for start in list_startend:
    start_index=start[0]
    end_index=None
    if len(start)==1:        
        end_index=None
    else:
#         start_index=start[0]
        end_index=start[1]
    # 获取header与body
#     header,body=None
    header,body=getHeaderBody(start_index+1,end_index)
    # 从header中获取name
    typhoon_name=header[7]
    
    print(f"当前body长度{len(body)}")
    for i in range(len(body)):
        if i>=len(body)-1:
            print("跳出本次循环")
            break
        temp_typhoon=body.iloc[i] 
        print(f"当前i:{i}")
        print(f"当前index:{start_index+1+i}")
        print(f"当前body:\n{body.iloc[i+1]}")
        typhoon_list.append(convert2Typhoon(body.iloc[i+1],typhoon_name))
    print('--------')

当前body长度10
当前i:0
当前index:1
当前body:
0    1956011806
1             1
2           135
3          1160
4          1004
5            12
Name: 2, dtype: int64
当前i:1
当前index:2
当前body:
0    1956011812
1             1
2           144
3          1166
4          1002
5            12
Name: 3, dtype: int64
当前i:2
当前index:3
当前body:
0    1956011818
1             1
2           152
3          1169
4          1002
5            15
Name: 4, dtype: int64
当前i:3
当前index:4
当前body:
0    1956011900
1             1
2           162
3          1169
4          1002
5            15
Name: 5, dtype: int64
当前i:4
当前index:5
当前body:
0    1956011906
1             1
2           173
3          1173
4          1002
5            15
Name: 6, dtype: int64
当前i:5
当前index:6
当前body:
0    1956011912
1             1
2           177
3          1178
4          1004
5            12
Name: 7, dtype: int64
当前i:6
当前index:7
当前body:
0    1956011918
1             1
2           180
3          1186
4          1006
5            12
Name: 8, dtype: i

Name: 306, dtype: int64
当前i:6
当前index:306
当前body:
0    1956080612
1             0
2           215
3          1079
4          1001
5            10
Name: 307, dtype: int64
当前i:7
当前index:307
当前body:
0    1956080618
1             0
2           225
3          1072
4          1001
5             9
Name: 308, dtype: int64
当前i:8
当前index:308
当前body:
0    1956080700
1             0
2           233
3          1068
4          1001
5             9
Name: 309, dtype: int64
跳出本次循环
--------
当前body长度13
当前i:0
当前index:311
当前body:
0    1956080712
1             0
2            90
3          1360
4          1004
5             0
Name: 312, dtype: int64
当前i:1
当前index:312
当前body:
0    1956080718
1             0
2            98
3          1352
4          1004
5             0
Name: 313, dtype: int64
当前i:2
当前index:313
当前body:
0    1956080800
1             0
2           102
3          1344
4          1004
5             0
Name: 314, dtype: int64
当前i:3
当前index:314
当前body:
0    1956080806
1             0
2           108

当前i:27
当前index:651
当前body:
0    1956092600
1             5
2           278
3          1288
4           957
5            50
Name: 652, dtype: int64
当前i:28
当前index:652
当前body:
0    1956092606
1             5
2           290
3          1303
4           958
5            50
Name: 653, dtype: int64
当前i:29
当前index:653
当前body:
0    1956092612
1             5
2           303
3          1322
4           959
5            45
Name: 654, dtype: int64
当前i:30
当前index:654
当前body:
0    1956092618
1             5
2           320
3          1343
4           964
5            45
Name: 655, dtype: int64
当前i:31
当前index:655
当前body:
0    1956092700
1             4
2           337
3          1368
4           970
5            40
Name: 656, dtype: int64
当前i:32
当前index:656
当前body:
0    1956092706
1             9
2           357
3          1404
4           980
5            30
Name: 657, dtype: int64
当前i:33
当前index:657
当前body:
0    1956092712
1             9
2           385
3          1449
4           990
5          

In [23]:
# 写入mongodb
index_save=0
for temp in typhoon_list:
    try:
        temp.save()
        index_save=index_save+1
        print(f'{index_save}写入成功')
    except ValidationError:
        print(f'出现错误的位置{index_save}')

1写入成功
2写入成功
3写入成功
4写入成功
5写入成功
6写入成功
7写入成功
8写入成功
9写入成功
10写入成功
11写入成功
12写入成功
13写入成功
14写入成功
15写入成功
16写入成功
17写入成功
18写入成功
19写入成功
20写入成功
21写入成功
22写入成功
23写入成功
24写入成功
25写入成功
26写入成功
27写入成功
28写入成功
29写入成功
30写入成功
31写入成功
32写入成功
33写入成功
34写入成功
35写入成功
36写入成功
37写入成功
38写入成功
39写入成功
40写入成功
41写入成功
42写入成功
43写入成功
44写入成功
45写入成功
46写入成功
47写入成功
48写入成功
49写入成功
50写入成功
51写入成功
52写入成功
53写入成功
54写入成功
55写入成功
56写入成功
57写入成功
58写入成功
59写入成功
60写入成功
61写入成功
62写入成功
63写入成功
64写入成功
65写入成功
66写入成功
67写入成功
68写入成功
69写入成功
70写入成功
71写入成功
72写入成功
73写入成功
74写入成功
75写入成功
76写入成功
77写入成功
78写入成功
79写入成功
80写入成功
81写入成功
82写入成功
83写入成功
84写入成功
85写入成功
86写入成功
87写入成功
88写入成功
89写入成功
90写入成功
91写入成功
92写入成功
93写入成功
94写入成功
95写入成功
96写入成功
97写入成功
98写入成功
99写入成功
100写入成功
101写入成功
102写入成功
103写入成功
104写入成功
105写入成功
106写入成功
107写入成功
108写入成功
109写入成功
110写入成功
111写入成功
112写入成功
113写入成功
114写入成功
115写入成功
116写入成功
117写入成功
118写入成功
119写入成功
120写入成功
121写入成功
122写入成功
123写入成功
124写入成功
125写入成功
126写入成功
127写入成功
128写入成功
129写入成功
130写入成功
131写入成功
132写入成功
133写入成功
134写入成功
135写入成功
136写入成功
137写入成功
138写入成功
139写

In [21]:
typhoon_list[1].code

'(nameless)'

In [24]:
typhoon_list[197].code

'(nameless)(-)1'

In [25]:
typhoon_list[678].code

'Jean'